In [47]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import gymnasium as gym
from collections import deque
import json

# === Инициализация среды ===
env = gym.make("CartPole-v0", render_mode="rgb_array")
state_dim = 4
n_actions = 2

# === Политическая сеть (исправленная) ===
class PolicyNetwork(nn.Module):  # Исправлено: название класса
    def __init__(self, state_dim, n_actions, hidden_size=64):
        super(PolicyNetwork, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(state_dim, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return torch.softmax(self.net(x), dim=-1)  # Вероятности действий

    def fit(self, elite_states, elite_actions):
        """Метод обучения: заменяет ручной оптимизационный цикл"""
        self.train()  # Включаем режим обучения
        optimizer.zero_grad()
        predicted_probs = self(elite_states)
        loss = criterion(predicted_probs, elite_actions)
        loss.backward()
        optimizer.step()
        return loss.item()  # Возвращаем loss для логирования

# === Агент и оптимизатор ===
agent = PolicyNetwork(state_dim, n_actions, hidden_size=64)
optimizer = optim.Adam(agent.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# === Функция генерации сессии (ваша задача — заполнить) ===
def generate_session_cartpole(t_max=300):
    states, actions = [], []
    total_reward = 0

    s, info = env.reset()

    for t in range(t_max):
        # YOUR CODE GOES HERE: choose action!
        state_tensor = torch.FloatTensor([s])  # Добавляем батч-измерение
        probs = agent(state_tensor).detach().numpy()[0]  # Получаем вероятности [2]
        a = np.random.choice(n_actions, p=probs)         # Сэмплируем действие

        new_s, r, done, truncated, info = env.step(a)

        # record sessions like you did before
        states.append(s.astype(dtype="object"))  # numpy array -> object для JSON
        actions.append(a)
        total_reward += r

        s = new_s
        if done:
            break

    return states, actions, total_reward

# === Выбор элитных сессий (уже работает, оставляем как есть) ===
def select_elites(states_batch, actions_batch, rewards_batch, percentile=70):
    reward_threshold = np.percentile(rewards_batch, percentile)
    elite_states, elite_actions = [], []

    for i in range(len(rewards_batch)):
        if rewards_batch[i] >= reward_threshold:
            elite_states.extend(states_batch[i])
            elite_actions.extend(actions_batch[i])

    return elite_states, elite_actions

# === Визуализация прогресса (заглушка) ===
def show_progress(rewards_batch, log, percentile, reward_range):
    mean_reward = np.mean(rewards_batch)
    best_reward = np.max(rewards_batch)
    log.append(mean_reward)
    print(f"Средняя награда: {mean_reward:.1f} | Лучшая: {best_reward:.1f} | Percentile: {percentile}")

# === Обучение ===
n_sessions = 100
percentile = 90
log = []

for i in range(200):
    # generate new sessions
    sessions = [generate_session_cartpole() for _ in range(n_sessions)]

    states_batch, actions_batch, rewards_batch = zip(*sessions)

    elite_states, elite_actions = select_elites(states_batch, actions_batch, rewards_batch, percentile)

    if len(elite_states) == 0:
        print(f"Нет элитных сессий. Средняя награда: {np.mean(rewards_batch):.1f}")
        continue

    # Преобразуем в тензоры
    elite_states = torch.FloatTensor(elite_states)
    elite_actions = torch.LongTensor(elite_actions)

    # Обучаем агента
    loss = agent.fit(elite_states, elite_actions)

    # Визуализируем прогресс
    if max(rewards_batch) > min(rewards_batch):
        show_progress(rewards_batch, log, percentile, reward_range=[0, np.max(rewards_batch)])

    # Проверка на победу
    if np.mean(rewards_batch) > 290:
        print("You Win!")
        break

# === Финальная генерация сессий для отправки ===
sessions = [generate_session_cartpole() for _ in range(n_sessions)]
states_batch, actions_batch, rewards_batch = zip(*sessions)
elite_states, elite_actions = select_elites(states_batch, actions_batch, rewards_batch, percentile)

# === Подготовка данных для JSON ===
sessions_to_send = []
for session in sessions:
    observations = [x.tolist() for x in session[0]]  # Исправлено: было 'x.tolist()ф' — опечатка!
    actions = [int(x) for x in session[1]]          # Приводим к int, если это numpy.int64
    sessions_to_send.append((observations, actions))

# === Сохранение в JSON ===
with open('sessions_to_send.json', 'w') as iofile:
    json.dump(sessions_to_send, iofile, ensure_ascii=True, indent=4)

print("\n=== Данные сохранены в sessions_to_send.json ===")

Средняя награда: 22.3 | Лучшая: 73.0 | Percentile: 90
Средняя награда: 28.3 | Лучшая: 93.0 | Percentile: 90
Средняя награда: 25.0 | Лучшая: 71.0 | Percentile: 90
Средняя награда: 30.6 | Лучшая: 117.0 | Percentile: 90
Средняя награда: 43.9 | Лучшая: 129.0 | Percentile: 90
Средняя награда: 52.5 | Лучшая: 155.0 | Percentile: 90
Средняя награда: 67.7 | Лучшая: 194.0 | Percentile: 90
Средняя награда: 70.2 | Лучшая: 206.0 | Percentile: 90
Средняя награда: 80.2 | Лучшая: 266.0 | Percentile: 90
Средняя награда: 81.8 | Лучшая: 159.0 | Percentile: 90
Средняя награда: 80.9 | Лучшая: 237.0 | Percentile: 90
Средняя награда: 79.8 | Лучшая: 182.0 | Percentile: 90
Средняя награда: 80.2 | Лучшая: 166.0 | Percentile: 90
Средняя награда: 85.9 | Лучшая: 160.0 | Percentile: 90
Средняя награда: 82.6 | Лучшая: 198.0 | Percentile: 90
Средняя награда: 89.0 | Лучшая: 225.0 | Percentile: 90
Средняя награда: 92.7 | Лучшая: 300.0 | Percentile: 90
Средняя награда: 85.7 | Лучшая: 185.0 | Percentile: 90
Средняя награ

In [ ]:
# DO NOT CHANGE THIS BLOCK
sessions = [generate_session_cartpole() for _ in range(n_sessions)]
states_batch, actions_batch, rewards_batch = zip(*sessions)
elite_states, elite_actions = select_elites(states_batch, actions_batch, rewards_batch, percentile)

sessions_to_send = []
for session in sessions:
  observations = [x.tolist() for x in session[0]]
  actions = [x.item() for x in session[1]]
  sessions_to_send.append((observations, actions))

import json
with open('sessions_to_send.json', 'w') as iofile:
  json.dump(sessions_to_send, iofile, ensure_ascii=True, indent=4)
# DO NOT CHANGE THIS BLOCK